In [ ]:
%run my_import

#uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
uri = "com-exp.local:5555"
#uri = "advantech.local:5555"

io = set_uri(uri)
io.debug = False
scan_ids = []


In [ ]:
scan_ids = reply_cmd(master_cmd_get_slave_descr).keys()
ids=list(scan_ids)
print(ids)
#assert(iD in ids)


In [ ]:
iD = ids[0]
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))
sdo_dict

In [ ]:
flash_param = ['fw_ver','analog_sample_freq_HZ','dac_tune_bitflag','smpl_num_pow2','dac_tune_hysteresis']
sdo_filter(flash_param, sdo_dict)

In [ ]:
read_sdo(['smpl_num_pow2'],[iD])

In [ ]:
write_sdo({'smpl_num_pow2': 6},[iD])

In [ ]:
def read_channel(bid:int, ch:int)->dict:
    write_sdo({'adc_idx':ch},[bid])
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def read_channel_offset(bid:int, ch:int, offset:int) :
    write_sdo({'adc_idx': ch,'dac_offset':offset},[bid])
    reply_cmd(ctrl_cmd_set_dac.set_bid(bid))
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

## read 1 channel

In [ ]:
read_channel(iD,0)

In [ ]:
read_channel_offset(iD,0,0x7FFF)

## read all 6 channels once

In [ ]:

adc_conv = list()
adc_raw = list()
for j in range(1):
    for i in range(6):
        yaml_msg = read_channel(iD,i)
        adc_raw.append(yaml_msg['adc_raw'])
        adc_conv.append(yaml_msg['adc_conv'])
        print(yaml_msg)
        time.sleep(0.1)
        
[hex(x) for x in adc_raw]

## reset DAC 

In [ ]:
for ch in range(6) :
    ret = read_channel_offset(iD,ch,0x7FFF)
    print(ret)

In [ ]:
def tune_adc_chan(bid:int, adc_idx:int) :
    dac_offset = 0x7FFF
    hister = 5
    tuned = False
    while ( not tuned ):
        yaml_msg = read_channel_offset(bid,adc_idx,dac_offset)
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        if ( adc_raw > 0x1FFF+hister ) :
            if ( dac_offset < 0xFFFF ) :
                #dac_offset += 2
                dac_offset -= 2
        elif ( adc_raw < 0x1FFF-hister ) :
            if ( dac_offset > 0 ) :
                #dac_offset -= 2
                dac_offset += 2
        else : 
            tuned = True
        time.sleep(0.001)
        yield(adc_raw, adc_conv, dac_offset)


In [ ]:
for ch in range(6) :
    print("--- {0} ---".format(ch))
    for adc_raw, adc_conv, offset in tune_adc_chan(iD,ch) :
        print("\r%f 0x%X 0x%X"%(adc_conv, adc_raw, offset), end="")
    print()

In [ ]:
cnt = 10
dac_offset = 0x7FFF
while (cnt) :
    cnt -= 1
    print(read_channel_offset(iD,0,dac_offset))
    dac_offset += 10


In [ ]:
io.doit(ctrl_cmd_dac_tune.set_bid(1))